### Simulated data

In [ ]:
import os
import glob
import numpy as np
import torch
from torch.utils.data import Dataset
from open3d import read_point_cloud

# import utils

In [ ]:
def find_valid_points(local_point_cloud):
    """
    find valid points in local point cloud
        invalid points have all zeros local coordinates
    local_point_cloud: <BxNxk> 
    valid_points: <BxN> indices  of valid point (0/1)
    """
    eps = 1e-6
    non_zero_coord = torch.abs(local_point_cloud) > eps
    valid_points = torch.sum(non_zero_coord, dim=-1)
    valid_points = valid_points > 0
    return valid_points


class SimulatedPointCloud(Dataset):
    def __init__(self, root, trans_by_pose=None):
        # trans_by_pose: <Bx3> pose
        self.root = os.path.expanduser(root)
        self._trans_by_pose = trans_by_pose
        file_list = glob.glob(os.path.join(self.root, '*pcd'))
        self.file_list = sorted(file_list)


#        self.pcds = [] # a list of open3d pcd objects 
        point_clouds = [] #a list of tensor <Lx2>
        for file in self.file_list:
            pcd = read_point_cloud(file)
#            self.pcds.append(pcd)
            current_point_cloud = np.asarray(pcd.points, dtype=np.float32)[:, 0:2]        
            point_clouds.append(current_point_cloud)

        point_clouds = np.asarray(point_clouds)
        try:
            self.point_clouds = torch.from_numpy(point_clouds) # <NxLx2>
        except Exception as e:
            print(e)

            # Handling the uniform size change across all point clouds
            NEW_SIZE = max([current_point_cloud.shape[0] for current_point_cloud in point_clouds]) + 1
            print('Aligning pt clouds to equal size of {}'.format(NEW_SIZE))

            np.random.seed(0)
            new_point_clouds = []
            for current_point_cloud in point_clouds:
                old_size = current_point_cloud.shape[0]
                delta_size = NEW_SIZE - old_size
                idx_list = np.random.randint(low=0, high=old_size, size=delta_size)
                delta_point_cloud = np.array([current_point_cloud[idx] for idx in idx_list])
                new_point_cloud = np.concatenate([current_point_cloud, delta_point_cloud])
                new_point_clouds.append(new_point_cloud)
            point_clouds = np.asarray(new_point_clouds)
            self.point_clouds = torch.from_numpy(point_clouds) # <NxLx2>

        self.valid_points = find_valid_points(self.point_clouds) # <NxL>

        # number of points in each point cloud
        self.n_obs = self.point_clouds.shape[1]

    def __getitem__(self, index):
        pcd = self.point_clouds[index,:,:]  # <Lx2>
        valid_points = self.valid_points[index,:]
        if self._trans_by_pose is not None:
            pcd = pcd.unsqueeze(0)  # <1XLx2>
            pose = self._trans_by_pose[index, :].unsqueeze(0)  # <1x3>
            pcd = utils.transform_to_global_2D(pose, pcd).squeeze(0)
        return pcd,valid_points

    def __len__(self):
        return len(self.point_clouds)

### Utils

#### Geometry utils

In [ ]:
import torch
import numpy as np
import open3d
from sklearn.neighbors import NearestNeighbors
import sys

In [ ]:
def transform_to_global_2D(pose, obs_local):
    """ 
    transform local point cloud to global frame
    row-based matrix product
    pose: <Bx3> each row represents <x,y,theta>
    obs_local: <BxLx2> 
    """
    L = obs_local.shape[1]
    # c0 is the loc of sensor in global coord. frame c0: <Bx2>
    c0, theta0 = pose[:, 0:2], pose[:, 2]
    c0 = c0.unsqueeze(1).expand(-1, L, -1)  # <BxLx2>

    cos = torch.cos(theta0).unsqueeze(-1).unsqueeze(-1)
    sin = torch.sin(theta0).unsqueeze(-1).unsqueeze(-1)
    R_transpose = torch.cat((cos, sin, -sin, cos), dim=1).reshape(-1, 2, 2)

    obs_global = torch.bmm(obs_local, R_transpose) + c0
    return obs_global

def transform_to_global_AVD(pose, obs_local):
    """
    transform obs local coordinate to global corrdinate frame
    :param pose: <Bx3> <x,z,theta> y = 0
    :param obs_local: <BxLx3> (unorganized) or <BxHxWx3> (organized)
    :return obs_global: <BxLx3> (unorganized) or <BxHxWx3> (organized)
    """
    is_organized = 1 if len(obs_local.shape) == 4 else 0
    b = obs_local.shape[0]
    if is_organized:
        H,W = obs_local.shape[1:3]
        obs_local = obs_local.view(b,-1,3) # <BxLx3>
    
    L = obs_local.shape[1]

    c0, theta0 = pose[:,0:2],pose[:,2] # c0 is the loc of sensor in global coord frame c0 <Bx2> <x,z>

    zero = torch.zeros_like(c0[:,:1])
    c0 = torch.cat((c0,zero),-1) # <Bx3> <x,z,y=0>
    c0 = c0[:,[0,2,1]] # <Bx3> <x,y=0,z>
    c0 = c0.unsqueeze(1).expand(-1,L,-1) # <BxLx3>
    
    cos = torch.cos(theta0).unsqueeze(-1).unsqueeze(-1)
    sin = torch.sin(theta0).unsqueeze(-1).unsqueeze(-1)
    zero = torch.zeros_like(sin)
    one = torch.ones_like(sin)
    
    R_y_transpose = torch.cat((cos,zero,-sin,zero,one,zero,sin,zero,cos),dim=1).reshape(-1,3,3)
    obs_global = torch.bmm(obs_local,R_y_transpose) + c0
    if is_organized:
        obs_global = obs_global.view(b,H,W,3)
    return obs_global


def rigid_transform_kD(A, B):
    """
    Find optimal transformation between two sets of corresponding points
    Adapted from: http://nghiaho.com/uploads/code/rigid_transform_3D.py_
    Args:
        A.B: <Nxk> each row represent a k-D points
    Returns:
        R: kxk
        t: kx1
        B = R*A+t
    """
    assert len(A) == len(B)
    N,k = A.shape
    
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    
    # centre the points
    AA = A - np.tile(centroid_A, (N, 1))
    BB = B - np.tile(centroid_B, (N, 1))

    H = np.matmul(np.transpose(AA) , BB)
    U, S, Vt = np.linalg.svd(H)
    R = np.matmul(Vt.T , U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
        Vt[k-1,:] *= -1
        R = np.matmul(Vt.T , U.T)

    t = np.matmul(-R,centroid_A.T) + centroid_B.T
    t = np.expand_dims(t,-1)
    return R, t

def estimate_normal_eig(data):
    """
    Computes the vector normal to the k-dimensional sample points
    """
    data -= np.mean(data,axis=0)
    data = data.T
    A = np.cov(data)
    w,v = np.linalg.eig(A)
    idx = np.argmin(w)
    v = v[:,idx]
    v /= np.linalg.norm(v,2)
    return v
    
def surface_normal(pc,n_neighbors=6):
    """
    Estimate point cloud surface normal
    Args:
        pc: Nxk matrix representing k-dimensional point cloud
    """
    
    n_points,k = pc.shape
    v = np.zeros_like(pc)
    
    # nn search
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='auto').fit(pc)
    _, indices = nbrs.kneighbors(pc)
    neighbor_points = pc[indices]
    for i in range(n_points):
        # estimate surface normal
        v_tmp = estimate_normal_eig(neighbor_points[i,])
        v_tmp[abs(v_tmp)<1e-5] = 0
        if v_tmp[0] < 0:
            v_tmp *= -1
        v[i,:] = v_tmp
    return v


def point2plane_metrics_2D(p,q,v):
    """
    Point-to-plane minimization
    Chen, Y. and G. Medioni. “Object Modelling by Registration of Multiple Range Images.” 
    Image Vision Computing. Butterworth-Heinemann . Vol. 10, Issue 3, April 1992, pp. 145-155.
    
    Args:
        p: Nx2 matrix, moving point locations
        q: Nx2 matrix, fixed point locations
        v:Nx2 matrix, fixed point normal
    Returns:
        R: 2x2 matrix
        t: 2x1 matrix
    """
    assert q.shape[1] == p.shape[1] == v.shape[1] == 2, 'points must be 2D'
    
    p,q,v = np.array(p),np.array(q),np.array(v)
    c = np.expand_dims(np.cross(p,v),-1)
    cn = np.concatenate((c,v),axis=1)  # [ci,nix,niy]
    C = np.matmul(cn.T,cn)
    if np.linalg.cond(C)>=1/sys.float_info.epsilon:
        # handle singular matrix
        raise ArithmeticError('Singular matrix')
    
#     print(C.shape)
    qp = q-p
    b = np.array([
        [(qp*cn[:,0:1]*v).sum()],
        [(qp*cn[:,1:2]*v).sum()],
        [(qp*cn[:,2:]*v).sum()],
    ])

    X = np.linalg.solve(C, b)
    cos_ = np.cos(X[0])[0]
    sin_ = np.sin(X[0])[0]
    R = np.array([
        [cos_,-sin_],
        [sin_,cos_]
    ])
    t = np.array(X[1:])
    return R,t

def icp(src,dst,nv=None,n_iter=100,init_pose=[0,0,0],torlerance=1e-6,metrics='point',verbose=False):
    '''
    Currently only works for 2D case
    Args:
        src: <Nx2> 2-dim moving points
        dst: <Nx2> 2-dim fixed points
        n_iter: a positive integer to specify the maxium nuber of iterations
        init_pose: [tx,ty,theta] initial transformation
        torlerance: the tolerance of registration error
        metrics: 'point' or 'plane'
        
    Return:
        src: transformed src points
        R: rotation matrix
        t: translation vector
        R*src + t
    '''
    n_src = src.shape[0]
    if metrics == 'plane' and nv is None:
        nv = surface_normal(dst)

    #src = np.matrix(src)
    #dst = np.matrix(dst)
    #Initialise with the initial pose estimation
    R_init = np.array([[np.cos(init_pose[2]),-np.sin(init_pose[2])],
                   [np.sin(init_pose[2]), np.cos(init_pose[2])] 
                      ])
    t_init = np.array([[init_pose[0]],
                   [init_pose[1]]
                      ])  
    
    #src =  R_init*src.T + t_init
    src = np.matmul(R_init,src.T) + t_init
    src = src.T
    
    R,t = R_init,t_init

    prev_err = np.inf
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(dst)
    for i in range(n_iter):
        # Find the nearest neighbours
        _, indices = nbrs.kneighbors(src)

        # Compute the transformation
        if metrics == 'point':
            R0,t0 = rigid_transform_kD(src,dst[indices[:,0]])
        elif metrics=='plane':
            try:
                R0,t0 = point2plane_metrics_2D(src,dst[indices[:,0]], nv[indices[:,0]]) 
            except ArithmeticError:
                print('Singular matrix')
                return src,R,t
        else:
            raise ValueError('metrics: {} not recognized.'.format(metrics))
        # Update dst and compute error
        src = np.matmul(R0,src.T) + t0
        src = src.T

        R = np.matmul(R0,R)
        t = np.matmul(R0,t) + t0
        #R = R0*R
        #t = R0*t + t0
        current_err = np.sqrt((np.array(src-dst[indices[:,0]])**2).sum()/n_src)

        if verbose:
            print('iter: {}, error: {}'.format(i,current_err))
            
        if  np.abs(current_err - prev_err) < torlerance:
            break
        else:
            prev_err = current_err
            
    return src,R,t


def compute_ate(output,target):
    """
    compute absolute trajectory error for avd dataset
    Args:
        output: <Nx3> predicted trajectory positions, where N is #scans
        target: <Nx3> ground truth trajectory positions
    Returns:
        trans_error: <N> absolute trajectory error for each pose
        output_aligned: <Nx3> aligned position in ground truth coord
    """
    R,t = rigid_transform_kD(output,target)
    output_aligned = np.matmul(R , output.T) + t
    output_aligned = output_aligned.T

    align_error = np.array(output_aligned - target)
    trans_error = np.sqrt(np.sum(align_error**2,1))
    
    ate = np.sqrt(np.dot(trans_error,trans_error) / len(trans_error))

    return ate,output_aligned

def remove_invalid_pcd(pcd):
    """
    remove invalid in valid points that have all-zero coordinates
    pcd: open3d pcd objective
    """
    pcd_np = np.asarray(pcd.points) # <Nx3>
    non_zero_coord = np.abs(pcd_np) > 1e-6 # <Nx3>
    valid_ind = np.sum(non_zero_coord,axis=-1)>0 #<N>
    valid_ind = list(np.nonzero(valid_ind)[0])
    valid_pcd = open3d.select_down_sample(pcd,valid_ind)
    return valid_pcd

def ang2mat(theta):
    c = np.cos(theta)
    s = np.sin(theta)
    R = np.array([[c,-s],[s,c]])
    return R

def cat_pose_2D(pose0,pose1):
    """
    pose0, pose1: <Nx3>, numpy array
    each row: <x,y,theta>
    """
    assert(pose0.shape==pose1.shape)
    n_pose = pose0.shape[0]
    pose_out = np.zeros_like(pose0) 
    for i in range(n_pose):
        R0 = ang2mat(pose0[i,-1])
        R1 = ang2mat(pose1[i,-1])
        t0 = np.expand_dims(pose0[i,:2],-1)
        t1 = np.expand_dims(pose1[i,:2],-1)
        
        R = np.matmul(R1,R0)
        theta = np.arctan2(R[1,0],R[0,0])
        t = np.matmul(R1,t0) + t1
        pose_out[i,:2] = t.T
        pose_out[i,2] = theta
    return pose_out

def convert_depth_map_to_pc(depth,fxy,cxy,max_depth=7000,depth_scale=2000):
    """
    create point cloud from depth map and camera instrinsic
    depth: <hxw> numpy array
    fxy: [fx,fy]
    cxy: [cx,cy]
    """
    fx,fy = fxy 
    cx,cy = cxy
    h,w = depth.shape
    
    c,r = np.meshgrid(range(1,w+1), range(1,h+1))
    invalid = depth >= max_depth
    depth[invalid] = 0

    z = depth / float(depth_scale)
    x = z * (c-cx) / fx
    y = z * (r-cy) / fy
    xyz = np.dstack((x,y,z)).astype(np.float32)
    return xyz

#### Open 3d utils

In [ ]:
import numpy as np
import open3d as o3d
import copy

In [ ]:
def transform_to_global_open3d(pose,local_pcd):
    pcd = copy.deepcopy(local_pcd)
    n_pcd = len(pcd)
    for i in range(n_pcd):
        tx,ty,theta = pose[i,:]
        cos,sin = np.cos(theta),np.sin(theta)
        trans = np.array([
                        [cos,-sin,0,tx],
                        [sin,cos,0,ty],
                        [0,0,1,0],
                        [0,0,0,1],
                        ])
        pcd[i].transform(trans) 
    return pcd


def np_to_pcd(xyz):
    """
    convert numpy array to point cloud object in open3d
    """
    xyz = xyz.reshape(-1,3)
    pcd = o3d.PointCloud()
    pcd.points = o3d.Vector3dVector(xyz)
    pcd.paint_uniform_color(np.random.rand(3,))
    return pcd


def load_obs_global_est(file_name):
    """
    load saved obs_global_est.npy file and convert to point cloud object
    """
    obs_global_est = np.load(file_name)
    n_pc = obs_global_est.shape[0]
    pcds = o3d.PointCloud()

    for i in range(n_pc):
        xyz = obs_global_est[i,:,:]
        current_pcd = np_to_pcd(xyz)
        pcds += current_pcd
    return pcds

#### Model utils

In [ ]:
import os
import json
import torch

In [ ]:
def save_opt(working_dir, opt):
    """
    Save option as a json file
    """
    opt = vars(opt)
    save_name = os.path.join(working_dir, 'opt.json')
    with open(save_name, 'wt') as f:
        json.dump(opt, f, indent=4, sort_keys=True)


def save_checkpoint(save_name, model, optimizer):
    state = {'state_dict': model.state_dict(),
             'optimizer': optimizer.state_dict()}
    torch.save(state, save_name)
    print('model saved to {}'.format(save_name))


def load_checkpoint(save_name, model, optimizer):
    state = torch.load(save_name)
    model.load_state_dict(state['state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(state['optimizer'])
    print('model loaded from {}'.format(save_name))


def load_opt_from_json(file_name):
    if os.path.isfile(file_name):
        with open(file_name,'rb') as f:
            opt_dict = json.load(f)
            return opt_dict
    else:
        raise FileNotFoundError("Can't find file: {}. Run training script first".format(file_name))

#### Visualization utils

In [ ]:
import os
from matplotlib import pyplot as plt
import torch
import numpy as np

In [ ]:
def save_opt(working_dir, opt):
    """
    Save option as a json file
    """
    opt = vars(opt)
    save_name = os.path.join(working_dir, 'opt.json')
    with open(save_name, 'wt') as f:
        json.dump(opt, f, indent=4, sort_keys=True)


def save_checkpoint(save_name, model, optimizer):
    state = {'state_dict': model.state_dict(),
             'optimizer': optimizer.state_dict()}
    torch.save(state, save_name)
    print('model saved to {}'.format(save_name))


def load_checkpoint(save_name, model, optimizer):
    state = torch.load(save_name)
    model.load_state_dict(state['state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(state['optimizer'])
    print('model loaded from {}'.format(save_name))


def load_opt_from_json(file_name):
    if os.path.isfile(file_name):
        with open(file_name,'rb') as f:
            opt_dict = json.load(f)
            return opt_dict
    else:
        raise FileNotFoundError("Can't find file: {}. Run training script first".format(file_name))

### BCE / Chamfer Losses

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class BCEWithLogitsLoss2(nn.Module):
    def __init__(self, weight=None, reduction='elementwise_mean'):
        super(BCEWithLogitsLoss2, self).__init__()
        self.reduction = reduction
        self.register_buffer('weight', weight)

    def forward(self, input, target):
        return bce_with_logits(input, target, weight=self.weight, reduction=self.reduction)


def bce_with_logits(input, target, weight=None, reduction='elementwise_mean'):
    """
    This function differs from F.binary_cross_entropy_with_logits in the way 
    that if weight is not None, the loss is normalized by weight
    """
    if not (target.size() == input.size()):
        raise ValueError("Target size ({}) must be the same as input size ({})".format(
            target.size(), input.size()))
    if weight is not None:
        if not (weight.size() == input.size()):
            raise ValueError("Weight size ({}) must be the same as input size ({})".format(
                weight.size(), input.size()))

    max_val = (-input).clamp(min=0)
    loss = input - input * target + max_val + \
        ((-max_val).exp() + (-input - max_val).exp()).log()

    if weight is not None:
        loss = loss * weight

    if reduction == 'none':
        return loss
    elif reduction == 'elementwise_mean':
        if weight is not None:
            # different from F.binary_cross_entropy_with_logits
            return loss.sum() / weight.sum()
        else:
            return loss.mean()
    else:
        return loss.sum()


def bce(pred, targets, weight=None):
    criternion = BCEWithLogitsLoss2(weight=weight)
    loss = criternion(pred, targets)
    return loss

In [ ]:
import torch
import torch.nn as nn

In [ ]:
INF = 1000000


class ChamfersDistance(nn.Module):
    '''
    Extensively search to compute the Chamfersdistance. 
    '''

    def forward(self, input1, input2, valid1=None, valid2=None):

        # input1, input2: BxNxK, BxMxK, K = 3
        B, N, K = input1.shape
        _, M, _ = input2.shape
        if valid1 is not None:
            # ignore invalid points
            valid1 = valid1.type(torch.float32)
            valid2 = valid2.type(torch.float32)

            invalid1 = 1 - valid1.unsqueeze(-1).expand(-1, -1, K)
            invalid2 = 1 - valid2.unsqueeze(-1).expand(-1, -1, K)

            input1 = input1 + invalid1 * INF * torch.ones_like(input1)
            input2 = input2 + invalid2 * INF * torch.ones_like(input2)

        # Repeat (x,y,z) M times in a row
        input11 = input1.unsqueeze(2)           # BxNx1xK
        input11 = input11.expand(B, N, M, K)    # BxNxMxK
        # Repeat (x,y,z) N times in a column
        input22 = input2.unsqueeze(1)           # Bx1xMxK
        input22 = input22.expand(B, N, M, K)    # BxNxMxK
        # compute the distance matrix
        D = input11 - input22                   # BxNxMxK
        D = torch.norm(D, p=2, dim=3)         # BxNxM

        dist0, _ = torch.min(D, dim=1)        # BxM
        dist1, _ = torch.min(D, dim=2)        # BxN

        if valid1 is not None:
            dist0 = torch.sum(dist0 * valid2, 1) / torch.sum(valid2, 1)
            dist1 = torch.sum(dist1 * valid1, 1) / torch.sum(valid1, 1)
        else:
            dist0 = torch.mean(dist0, 1)
            dist1 = torch.mean(dist1, 1)

        loss = dist0 + dist1  # B
        loss = torch.mean(loss)                             # 1
        return loss


def registration_loss(obs, valid_obs=None):
    """
    Registration consistency
    obs: <BxLx2> a set of obs frame in the same coordinate system
    select of frame as reference (ref_id) and the rest as target
    compute chamfer distance between each target frame and reference

    valid_obs: <BxL> indics of valid points in obs
    """
    criternion = ChamfersDistance()
    bs = obs.shape[0]
    ref_id = 0
    ref_map = obs[ref_id, :, :].unsqueeze(0).expand(bs - 1, -1, -1)
    valid_ref = valid_obs[ref_id, :].unsqueeze(0).expand(bs - 1, -1)

    tgt_list = list(range(bs))
    tgt_list.pop(ref_id)
    tgt_map = obs[tgt_list, :, :]
    valid_tgt = valid_obs[tgt_list, :]

    loss = criternion(ref_map, tgt_map, valid_ref, valid_tgt)
    return loss


def chamfer_loss(obs, valid_obs=None, seq=2):
    bs = obs.shape[0]
    total_step = bs - seq + 1
    loss = 0.
    for step in range(total_step):
        current_obs = obs[step:step + seq]
        current_valid_obs = valid_obs[step:step + seq]

        current_loss = registration_loss(current_obs, current_valid_obs)
        loss = loss + current_loss

    loss = loss / total_step
    return loss

### Deepmapping / Network Models

In [ ]:
from copy import deepcopy
import numpy as np
import torch
import torch.nn as nn
# from .networks import LocNetReg2D, LocNetRegAVD, MLP
# from utils import transform_to_global_2D, transform_to_global_AVD

In [ ]:
def get_M_net_inputs_labels(occupied_points, unoccupited_points):
    """
    get global coord (occupied and unoccupied) and corresponding labels
    """
    n_pos = occupied_points.shape[1]
    inputs = torch.cat((occupied_points, unoccupited_points), 1)
    bs, N, _ = inputs.shape

    gt = torch.zeros([bs, N, 1], device=occupied_points.device)
    gt.requires_grad_(False)
    gt[:, :n_pos, :] = 1
    return inputs, gt


def sample_unoccupied_point(local_point_cloud, n_samples):
    """
    sample unoccupied points along rays in local point cloud
    sensor located at origin
    local_point_cloud: <BxNxk>
    n_samples: number of samples on each ray
    """
    bs, L, k = local_point_cloud.shape
    unoccupied = torch.zeros(bs, L * n_samples, k,
                             device=local_point_cloud.device)
    for idx in range(1, n_samples + 1):
        fac = torch.rand(1).item()
        unoccupied[:, (idx - 1) * L:idx * L, :] = local_point_cloud * fac
    return unoccupied

class DeepMapping2D(nn.Module):
    def __init__(self, loss_fn, n_obs=256, n_samples=19, dim=[2, 64, 512, 512, 256, 128, 1]):
        super(DeepMapping2D, self).__init__()
        self.n_obs = n_obs
        self.n_samples = n_samples
        self.loss_fn = loss_fn
        self.loc_net = LocNetReg2D(n_points=n_obs, out_dims=3)
        self.occup_net = MLP(dim)

    def forward(self, obs_local,valid_points):
        # obs_local: <BxLx2>
        self.obs_local = deepcopy(obs_local)
        self.valid_points = valid_points

        self.pose_est = self.loc_net(self.obs_local)

        self.obs_global_est = transform_to_global_2D(
            self.pose_est, self.obs_local)

        if self.training:
            self.unoccupied_local = sample_unoccupied_point(
                self.obs_local, self.n_samples)
            self.unoccupied_global = transform_to_global_2D(
                self.pose_est, self.unoccupied_local)

            inputs, self.gt = get_M_net_inputs_labels(
                self.obs_global_est, self.unoccupied_global)
            self.occp_prob = self.occup_net(inputs)
            loss = self.compute_loss()
            return loss

    def compute_loss(self):
        valid_unoccupied_points = self.valid_points.repeat(1, self.n_samples)
        bce_weight = torch.cat(
            (self.valid_points, valid_unoccupied_points), 1).float()
        # <Bx(n+1)Lx1> same as occp_prob and gt
        bce_weight = bce_weight.unsqueeze(-1)

        if self.loss_fn.__name__ == 'bce_ch':
            loss = self.loss_fn(self.occp_prob, self.gt, self.obs_global_est,
                                self.valid_points, bce_weight, seq=4, gamma=0.1)  # BCE_CH
        elif self.loss_fn.__name__ == 'bce':
            loss = self.loss_fn(self.occp_prob, self.gt, bce_weight)  # BCE
        return loss

class DeepMapping_AVD(nn.Module):
    #def __init__(self, loss_fn, n_samples=35, dim=[3, 256, 256, 256, 256, 256, 256, 1]):
    def __init__(self, loss_fn, n_samples=35, dim=[3, 64, 512, 512, 256, 128, 1]):
        super(DeepMapping_AVD, self).__init__()
        self.n_samples = n_samples
        self.loss_fn = loss_fn
        self.loc_net = LocNetRegAVD(out_dims=3) # <x,z,theta> y=0
        self.occup_net = MLP(dim)

    def forward(self, obs_local,valid_points):
        # obs_local: <BxHxWx3> 
        # valid_points: <BxHxW>
        
        self.obs_local = deepcopy(obs_local)
        self.valid_points = valid_points
        self.pose_est = self.loc_net(self.obs_local)

        bs = obs_local.shape[0]
        self.obs_local = self.obs_local.view(bs,-1,3)
        self.valid_points = self.valid_points.view(bs,-1)
        
        self.obs_global_est = transform_to_global_AVD(
            self.pose_est, self.obs_local)

        if self.training:
            self.unoccupied_local = sample_unoccupied_point(
                self.obs_local, self.n_samples)
            self.unoccupied_global = transform_to_global_AVD(
                self.pose_est, self.unoccupied_local)

            inputs, self.gt = get_M_net_inputs_labels(
                self.obs_global_est, self.unoccupied_global)
            self.occp_prob = self.occup_net(inputs)
            loss = self.compute_loss()
            return loss

    def compute_loss(self):
        valid_unoccupied_points = self.valid_points.repeat(1, self.n_samples)
        bce_weight = torch.cat(
            (self.valid_points, valid_unoccupied_points), 1).float()
        # <Bx(n+1)Lx1> same as occp_prob and gt
        bce_weight = bce_weight.unsqueeze(-1)

        if self.loss_fn.__name__ == 'bce_ch':
            loss = self.loss_fn(self.occp_prob, self.gt, self.obs_global_est,
                                self.valid_points, bce_weight, seq=2, gamma=0.9)  # BCE_CH
        elif self.loss_fn.__name__ == 'bce':
            loss = self.loss_fn(self.occp_prob, self.gt, bce_weight)  # BCE
        return loss

In [ ]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def get_and_init_FC_layer(din, dout):
    li = nn.Linear(din, dout)
    nn.init.xavier_uniform_(
       li.weight.data, gain=nn.init.calculate_gain('relu'))
    li.bias.data.fill_(0.)
    return li


def get_MLP_layers(dims, doLastRelu):
    layers = []
    for i in range(1, len(dims)):
        layers.append(get_and_init_FC_layer(dims[i - 1], dims[i]))
        if i == len(dims) - 1 and not doLastRelu:
            continue
        layers.append(nn.ReLU())
    return layers


class PointwiseMLP(nn.Sequential):
    def __init__(self, dims, doLastRelu=False):
        layers = get_MLP_layers(dims, doLastRelu)
        super(PointwiseMLP, self).__init__(*layers)


class MLP(nn.Module):
    def __init__(self, dims):
        super(MLP, self).__init__()
        self.mlp = PointwiseMLP(dims, doLastRelu=False)

    def forward(self, x):
        return self.mlp.forward(x)


class ObsFeat2D(nn.Module):
    """Feature extractor for 1D organized point clouds"""

    def __init__(self, n_points, n_out=1024):
        super(ObsFeat2D, self).__init__()
        self.n_out = n_out
        k = 3
        p = int(np.floor(k / 2)) + 2
        self.conv1 = nn.Conv1d(2, 64, kernel_size=k, padding=p, dilation=3)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=k, padding=p, dilation=3)
        self.conv3 = nn.Conv1d(
            128, self.n_out, kernel_size=k, padding=p, dilation=3)
        self.mp = nn.MaxPool1d(n_points)

    def forward(self, x):
        assert(x.shape[1] == 2), "the input size must be <Bx2xL> "

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)
        x = self.mp(x)
        x = x.view(-1, self.n_out)  # <Bx1024>
        return x


class ObsFeatAVD(nn.Module):
    """Feature extractor for 2D organized point clouds"""
    def __init__(self, n_out=1024):
        super(ObsFeatAVD, self).__init__()
        self.n_out = n_out
        k = 3
        p = int(np.floor(k / 2)) + 2
        self.conv1 = nn.Conv2d(3,64,kernel_size=k,padding=p,dilation=3)
        self.conv2 = nn.Conv2d(64,128,kernel_size=k,padding=p,dilation=3)
        self.conv3 = nn.Conv2d(128,256,kernel_size=k,padding=p,dilation=3)
        self.conv4 = nn.Conv2d(256,self.n_out,kernel_size=k,padding=p,dilation=3)
        self.amp = nn.AdaptiveMaxPool2d(1)

    def forward(self, x):
        assert(x.shape[1]==3),"the input size must be <Bx3xHxW> "
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))        
        x = F.relu(self.conv3(x))
        x = self.conv4(x)
        x = self.amp(x) 
        x = x.view(-1,self.n_out) #<Bxn_out>
        return x


class LocNetReg2D(nn.Module):
    def __init__(self, n_points, out_dims):
        super(LocNetReg2D, self).__init__()
        self.obs_feat_extractor = ObsFeat2D(n_points)
        n_in = self.obs_feat_extractor.n_out
        self.fc = MLP([n_in, 512, 256, out_dims])

    def forward(self, obs):
        obs = obs.transpose(1, 2)
        obs_feat = self.obs_feat_extractor(obs)
        obs = obs.transpose(1, 2)

        x = self.fc(obs_feat)
        return x


class LocNetRegAVD(nn.Module):
    def __init__(self, out_dims):
        super(LocNetRegAVD, self).__init__()
        self.obs_feat_extractor = ObsFeatAVD()
        n_in = self.obs_feat_extractor.n_out
        self.fc = MLP([n_in, 512, 256, out_dims])

    def forward(self, obs):
        # obs: <BxHxWx3>
        bs = obs.shape[0]
        obs = obs.permute(0,3,1,2) # <Bx3xHxW>
        obs_feat = self.obs_feat_extractor(obs)
        obs = obs.permute(0,2,3,1)

        x = self.fc(obs_feat)
        return x

### Train 2d

In [ ]:
import set_path
import os
import argparse
import functools

import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

import utils
import loss
# from models import DeepMapping2D
# from dataset_loader import SimulatedPointCloud

In [ ]:
print = functools.partial(print,flush=True)

torch.backends.cudnn.deterministic = True
torch.manual_seed(999)parser = argparse.ArgumentParser()
parser.add_argument('--name',type=str,default='test',help='experiment name')
parser.add_argument('-m','--metric',type=str,default='point',choices=['point','plane'] ,help='minimization metric')
parser.add_argument('-d','--data_dir',type=str,default='../data/2D/',help='dataset path')
parser.add_argument('-r','--radius',type=float,default=0.02)
opt = parser.parse_args()
print(opt.radius)

checkpoint_dir = os.path.join('../results/2D',opt.name)
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
utils.save_opt(checkpoint_dir,opt)

dataset = SimulatedPointCloud(opt.data_dir)
local_pcds = dataset.pcds[:]
n_pc = len(local_pcds)

#"""
# remove invalid points
local_pcds = [utils.remove_invalid_pcd(x) for x in local_pcds]

if opt.metric == 'point':
    metric = open3d.TransformationEstimationPointToPoint() 
else:
    metric = open3d.TransformationEstimationPointToPlane() 
    for idx in range(n_pc):
        open3d.estimate_normals(local_pcds[idx],search_param = open3d.KDTreeSearchParamHybrid(radius=opt.radius,max_nn=10))


pose_est = np.zeros((n_pc,3),dtype=np.float32)
print('running icp')
for idx in range(n_pc-1):
    dst = local_pcds[idx]
    src = local_pcds[idx+1]
    result_icp = open3d.registration_icp(src,dst,opt.radius,estimation_method=metric)

    R0 = result_icp.transformation[:2,:2]
    t0 = result_icp.transformation[:2,3:]
    if idx == 0: 
        R_cum = R0
        t_cum = t0
    else:
        R_cum = np.matmul(R_cum , R0)
        t_cum = np.matmul(R_cum,t0) + t_cum
    
    pose_est[idx+1,:2] = t_cum.T
    pose_est[idx+1,2] = np.arctan2(R_cum[1,0],R_cum[0,0]) 

save_name = os.path.join(checkpoint_dir,'pose_est.npy')
np.save(save_name,pose_est)

# plot point cloud in global frame

print('saving results')
global_pcds = utils.transform_to_global_open3d(pose_est,local_pcds)
utils.save_global_point_cloud_open3d(global_pcds,pose_est,checkpoint_dir)

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--name',type=str,default='test',help='experiment name')
parser.add_argument('-e','--n_epochs',type=int,default=1000,help='number of epochs')
parser.add_argument('-b','--batch_size',type=int,default=32,help='batch_size')
parser.add_argument('-l','--loss',type=str,default='bce_ch',help='loss function')
parser.add_argument('-n','--n_samples',type=int,default=19,help='number of sampled unoccupied points along rays')
parser.add_argument('--lr',type=float,default=0.001,help='learning rate')
parser.add_argument('-d','--data_dir',type=str,default='../data/2D/',help='dataset path')
parser.add_argument('-m','--model', type=str, default=None,help='pretrained model name')
parser.add_argument('-i','--init', type=str, default=None,help='init pose')
parser.add_argument('--log_interval',type=int,default=10,help='logging interval of saving results')

opt = parser.parse_args()

checkpoint_dir = os.path.join('../results/2D',opt.name)
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
utils.save_opt(checkpoint_dir,opt)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('loading dataset')
if opt.init is not None:
    init_pose_np = np.load(opt.init)
    init_pose = torch.from_numpy(init_pose_np)
else:
    init_pose = None
dataset = SimulatedPointCloud(opt.data_dir,init_pose)
loader = DataLoader(dataset,batch_size=opt.batch_size,shuffle=False)

loss_fn = eval('loss.'+opt.loss)

print('creating model')
model = DeepMapping2D(loss_fn=loss_fn,n_obs=dataset.n_obs, n_samples=opt.n_samples).to(device)
optimizer = optim.Adam(model.parameters(),lr=opt.lr)

if opt.model is not None:
    utils.load_checkpoint(opt.model,model,optimizer)

print('start training')
for epoch in range(opt.n_epochs):

    training_loss= 0.0
    model.train()

    for index,(obs_batch,valid_pt) in enumerate(loader):
        obs_batch = obs_batch.to(device)
        valid_pt = valid_pt.to(device)
        loss = model(obs_batch,valid_pt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        training_loss += loss.item()
    
    training_loss_epoch = training_loss/len(loader)

    if (epoch+1) % opt.log_interval == 0:
        print('[{}/{}], training loss: {:.4f}'.format(
            epoch+1,opt.n_epochs,training_loss_epoch))

        obs_global_est_np = []
        pose_est_np = []
        with torch.no_grad():
            model.eval()
            for index,(obs_batch,valid_pt) in enumerate(loader):
                obs_batch = obs_batch.to(device)
                valid_pt = valid_pt.to(device)
                model(obs_batch,valid_pt)

                obs_global_est_np.append(model.obs_global_est.cpu().detach().numpy())
                pose_est_np.append(model.pose_est.cpu().detach().numpy())
            
            pose_est_np = np.concatenate(pose_est_np)
            if init_pose is not None:
                pose_est_np = utils.cat_pose_2D(init_pose_np,pose_est_np)

            save_name = os.path.join(checkpoint_dir,'model_best.pth')
            utils.save_checkpoint(save_name,model,optimizer)

            obs_global_est_np = np.concatenate(obs_global_est_np)
            kwargs = {'e':epoch+1}
            valid_pt_np = dataset.valid_points.cpu().detach().numpy()
            utils.plot_global_point_cloud(obs_global_est_np,pose_est_np,valid_pt_np,checkpoint_dir,**kwargs)

            #save_name = os.path.join(checkpoint_dir,'obs_global_est.npy')
            #np.save(save_name,obs_global_est_np)

            save_name = os.path.join(checkpoint_dir,'pose_est.npy')
            np.save(save_name,pose_est_np)

### Evaluation


In [ ]:
import set_path
import os
import argparse
import functools

import numpy as np
import scipy.io as sio

import utils

In [ ]:
print = functools.partial(print,flush=True)

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('-c','--checkpoint_dir',type=str,required=True,help='path to results folder')
opt = parser.parse_args()
saved_json_file = os.path.join(opt.checkpoint_dir,'opt.json')
train_opt = utils.load_opt_from_json(saved_json_file)
name = train_opt['name']
data_dir = train_opt['data_dir']

# load ground truth poses
gt_file = os.path.join(data_dir,'gt_pose.mat')
gt_pose = sio.loadmat(gt_file)
gt_pose = gt_pose['pose']
gt_location = gt_pose[:,:2]

# load predicted poses
pred_file = os.path.join(opt.checkpoint_dir,'pose_est.npy')
pred_pose = np.load(pred_file)
pred_location = pred_pose[:,:2] * 512 # denormalization, tbd

# compute absolute trajectory error (ATE)
ate,aligned_location = utils.compute_ate(pred_location,gt_location) 
print('{}, ate: {}'.format(name,ate))

### Incremental ICP

In [ ]:
import set_path
import os
import argparse
import functools

import torch
import numpy as np

# import utils
# from dataset_loader import SimulatedPointCloud

In [ ]:
print = functools.partial(print,flush=True)

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--name',type=str,default='test',help='experiment name')
parser.add_argument('-m','--metric',type=str,default='point',choices=['point','plane'] ,help='minimization metric')
parser.add_argument('-d','--data_dir',type=str,default='../data/2D/',help='dataset path')
opt = parser.parse_args()

checkpoint_dir = os.path.join('../results/2D',opt.name)
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
utils.save_opt(checkpoint_dir,opt)

dataset = SimulatedPointCloud(opt.data_dir)
n_pc = len(dataset)

pose_est = np.zeros((n_pc,3),dtype=np.float32)
print('running icp')
for idx in range(n_pc-1):
    dst,valid_dst = dataset[idx] 
    src,valid_src = dataset[idx+1]
    
    dst = dst[valid_dst,:].numpy()
    src = src[valid_src,:].numpy()

    _,R0,t0 = utils.icp(src,dst,metrics=opt.metric)
    if idx == 0: 
        R_cum = R0
        t_cum = t0
    else:
        R_cum = np.matmul(R_cum , R0)
        t_cum = np.matmul(R_cum,t0) + t_cum
    
    pose_est[idx+1,:2] = t_cum.T
    pose_est[idx+1,2] = np.arctan2(R_cum[1,0],R_cum[0,0]) 

save_name = os.path.join(checkpoint_dir,'pose_est.npy')
np.save(save_name,pose_est)

print('saving results')
pose_est = torch.from_numpy(pose_est)
local_pc,valid_id = dataset[:]
global_pc = utils.transform_to_global_2D(pose_est,local_pc)
utils.plot_global_point_cloud(global_pc,pose_est,valid_id,checkpoint_dir)

In [ ]:
import set_path
import os
import argparse
import functools

import torch
import numpy as np
import open3d

# import utils
# from dataset_loader import SimulatedPointCloud

In [ ]:
print = functools.partial(print,flush=True)

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--name',type=str,default='test',help='experiment name')
parser.add_argument('-m','--metric',type=str,default='point',choices=['point','plane'] ,help='minimization metric')
parser.add_argument('-d','--data_dir',type=str,default='../data/2D/',help='dataset path')
parser.add_argument('-r','--radius',type=float,default=0.02)
opt = parser.parse_args()
print(opt.radius)

checkpoint_dir = os.path.join('../results/2D',opt.name)
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
utils.save_opt(checkpoint_dir,opt)

dataset = SimulatedPointCloud(opt.data_dir)
local_pcds = dataset.pcds[:]
n_pc = len(local_pcds)

#"""
# remove invalid points
local_pcds = [utils.remove_invalid_pcd(x) for x in local_pcds]

if opt.metric == 'point':
    metric = open3d.TransformationEstimationPointToPoint() 
else:
    metric = open3d.TransformationEstimationPointToPlane() 
    for idx in range(n_pc):
        open3d.estimate_normals(local_pcds[idx],search_param = open3d.KDTreeSearchParamHybrid(radius=opt.radius,max_nn=10))


pose_est = np.zeros((n_pc,3),dtype=np.float32)
print('running icp')
for idx in range(n_pc-1):
    dst = local_pcds[idx]
    src = local_pcds[idx+1]
    result_icp = open3d.registration_icp(src,dst,opt.radius,estimation_method=metric)

    R0 = result_icp.transformation[:2,:2]
    t0 = result_icp.transformation[:2,3:]
    if idx == 0: 
        R_cum = R0
        t_cum = t0
    else:
        R_cum = np.matmul(R_cum , R0)
        t_cum = np.matmul(R_cum,t0) + t_cum
    
    pose_est[idx+1,:2] = t_cum.T
    pose_est[idx+1,2] = np.arctan2(R_cum[1,0],R_cum[0,0]) 

save_name = os.path.join(checkpoint_dir,'pose_est.npy')
np.save(save_name,pose_est)

# plot point cloud in global frame

print('saving results')
global_pcds = utils.transform_to_global_open3d(pose_est,local_pcds)
utils.save_global_point_cloud_open3d(global_pcds,pose_est,checkpoint_dir)